## **1-1. 희소 표현 기반 임베딩**

**원-핫 인코딩 적용**

In [1]:
import pandas as pd
import torch

class2 = pd.read_csv("./data/class2.csv") # 데이터셋을 메모리로 로딩

from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

train_x = label_encoder.fit_transform(class2['class2'])
train_x

array([2, 2, 1, 0, 1, 0])

## **1-2. 횟수 기반 임베딩**

### **카운터 벡터**

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is last chance.',
    'and if you do not have this chance.',
    'you will never get any chance.',
    'will you do get this one?',
    'please, get this chance',
]

vect = CountVectorizer()
vect.fit(corpus)
vect.vocabulary_

{'this': 13,
 'is': 7,
 'last': 8,
 'chance': 2,
 'and': 0,
 'if': 6,
 'you': 15,
 'do': 3,
 'not': 10,
 'have': 5,
 'will': 14,
 'never': 9,
 'get': 4,
 'any': 1,
 'one': 11,
 'please': 12}

In [3]:
### 배열 변환

vect.transform(['you will never get any chance.']).toarray()

array([[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]], dtype=int64)

In [4]:
## 불용어를 제거한 countvector

vect = CountVectorizer(stop_words = ["and", "is", "please", "this"]).fit(corpus)
vect.vocabulary_

{'last': 6,
 'chance': 1,
 'if': 5,
 'you': 11,
 'do': 2,
 'not': 8,
 'have': 4,
 'will': 10,
 'never': 7,
 'get': 3,
 'any': 0,
 'one': 9}

### **TF-IDF**

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

doc = ['I like machine learning', 'I love deep learning', 'I run everyday']

tfidf_vectorizer = TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc)
doc_distance = (tfidf_matrix * tfidf_matrix.T)

print ('유사도를 위한', str(doc_distance.get_shape()[0]), 'x', str(doc_distance.get_shape()[1]), 'matrix를 만들었습니다.')
print(doc_distance.toarray())

유사도를 위한 3 x 3 matrix를 만들었습니다.
[[1.       0.224325 0.      ]
 [0.224325 1.       0.      ]
 [0.       0.       1.      ]]


## **1-3. 예측 기반 임베딩**

### **워드투벡터**

In [ ]:
!pip install nltk

In [ ]:
import nltk

nltk.download("popular")

In [8]:
## 데이터셋 로딩 & 토큰화

from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
warnings.filterwarnings(action = 'ignore') 
import gensim 
from gensim.models import Word2Vec
  
sample = open("./data/peter.txt", "r", encoding = 'UTF8')
s = sample.read() 
  
f = s.replace("\n", " ")
data = [] 
  
for i in sent_tokenize(f): # 로딩된 파일의 각 문장마다 반복
    temp = [] 
    for j in word_tokenize(i): # 문장을 단어로 토큰화
        temp.append(j.lower()) # 토큰화된 단어를 소문자로 변환 후 temp에 저장
    data.append(temp) 

data

[['once',
  'upon',
  'a',
  'time',
  'in',
  'london',
  ',',
  'the',
  'darlings',
  'went',
  'out',
  'to',
  'a',
  'dinner',
  'party',
  'leaving',
  'their',
  'three',
  'children',
  'wendy',
  ',',
  'jhon',
  ',',
  'and',
  'michael',
  'at',
  'home',
  '.'],
 ['after',
  'wendy',
  'had',
  'tucked',
  'her',
  'younger',
  'brothers',
  'jhon',
  'and',
  'michael',
  'to',
  'bed',
  ',',
  'she',
  'went',
  'to',
  'read',
  'a',
  'book',
  '.'],
 ['she', 'heard', 'a', 'boy', 'sobbing', 'outside', 'her', 'window', '.'],
 ['he', 'was', 'flying', '.'],
 ['there', 'was', 'little', 'fairy', 'fluttering', 'around', 'him', '.'],
 ['wendy', 'opened', 'the', 'window', 'to', 'talk', 'to', 'him', '.'],
 ['“', 'hello', '!'],
 ['who', 'are', 'you', '?'],
 ['why', 'are', 'you', 'crying', '”', ',', 'wendy', 'asked', 'him', '.'],
 ['“', 'my', 'name', 'is', 'peter', 'pan', '.'],
 ['my',
  'shadow',
  'wouldn',
  '’',
  't',
  'stock',
  'to',
  'me.',
  '”',
  ',',
  'he',
  'rep

### **CBOW**

In [9]:
## CBOW 적용 후 단어 간 유사성 확인
# peter vs wendy

model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              vector_size = 100, window = 5, sg = 0)    
print("Cosine similarity between 'peter' " +
                 "wendy' - CBOW : ", 
      model1.wv.similarity('wendy', 'wendy'))

model1 = gensim.models.Word2Vec(data, min_count = 1,  
                              vector_size = 100, window = 5, sg = 0)

Cosine similarity between 'peter' wendy' - CBOW :  1.0


- 코사인 유사도가 100%로 나타나고 있음

In [10]:
# peter vs hook

print("Cosine similarity between 'peter' " + "hook' - CBOW : ", 
      model1.wv.similarity('peter', 'hook')) 

Cosine similarity between 'peter' hook' - CBOW :  0.027709898


### **skip-gram**

In [12]:
# peter vs wendy

model2 = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, 
                                             window = 5, sg = 1)
print("Cosine similarity between 'peter' " +
          "wendy' - Skip Gram : ", 
    model2.wv.similarity('peter', 'wendy'))

Cosine similarity between 'peter' wendy' - Skip Gram :  0.40088683


In [13]:
# peter vs hook

print("Cosine similarity between 'peter' " +
            "hook' - Skip Gram : ", 
      model2.wv.similarity('peter', 'hook')) 

Cosine similarity between 'peter' hook' - Skip Gram :  0.52016735


- CBOW와는 유사도가 다르게 측정됨

### **패스트텍스트**

In [15]:
from gensim.test.utils import common_texts
from gensim.models import FastText

model = FastText('./data/peter.txt', vector_size = 4, window = 3, min_count = 1, epochs = 10)

In [16]:
# peter, wendy에 대한 코사인 유사도

sim_score = model.wv.similarity('peter', 'wendy')
print(sim_score)

0.45924556


In [17]:
# peter, hook에 대한 코사인 유사도

sim_score = model.wv.similarity('peter', 'hook')
print(sim_score)

0.043825187


**사전 훈련된 패스트텍스트 모델 사용하기**

In [18]:
from __future__ import print_function
from gensim.models import KeyedVectors # gensim은 자연어를 벡터로 변환하는 데 필요한 편의 기능을 제공함

model_kr = KeyedVectors.load_word2vec_format('./data/wiki.ko.vec') # 파일을 메모리로 불러오기

In [19]:
# '노략'과 유사한 단어의 유사도 확인

find_similar_to = '노력'

for similar_word in model_kr.similar_by_word(find_similar_to):
    print("Word: {0}, Similarity: {1:.2f}".format(
        similar_word[0], similar_word[1]))

Word: 노력함, Similarity: 0.80
Word: 노력중, Similarity: 0.75
Word: 노력만, Similarity: 0.72
Word: 노력과, Similarity: 0.71
Word: 노력의, Similarity: 0.69
Word: 노력가, Similarity: 0.69
Word: 노력이나, Similarity: 0.69
Word: 노력없이, Similarity: 0.68
Word: 노력맨, Similarity: 0.68
Word: 노력보다는, Similarity: 0.68


In [20]:
# '동물', '육식동물'에는 긍정적이지만 '사람'에게는 부정적인 단어의 유사도 확인

similarities = model_kr.most_similar(positive = ['동물', '육식동물'], negative = ['사람'])
print(similarities)

[('초식동물', 0.7804122567176819), ('거대동물', 0.7547270655632019), ('육식동물의', 0.7547166347503662), ('유두동물', 0.753511369228363), ('반추동물', 0.7470757961273193), ('독동물', 0.7466291785240173), ('육상동물', 0.746031641960144), ('유즐동물', 0.7450903654098511), ('극피동물', 0.7449344396591187), ('복모동물', 0.742434561252594)]


## **1-4. 횟수/예측 기반 임베딩**

### **글로브**

In [25]:
## 라이브러리 호출 및 데이터셋 로딩

import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('C:/waterbean/jupyter/pytorch_textbook/Chapter10/data/glove.6B.100d.txt')                                       
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt") # 글로브 데이터 > Word2Vec
glove2word2vec(glove_file, word2vec_glove_file)

(400000, 100)

In [26]:
# 'bill'과 유사한 단어의 리스트를 반환

model = KeyedVectors.load_word2vec_format(word2vec_glove_file) # word2vec 형태로 가져오기
model.most_similar('bill') # 해당 단어를 기준으로 가장 유사한 단어들의 리스트 보여주기

[('legislation', 0.8072139620780945),
 ('proposal', 0.730686366558075),
 ('senate', 0.7142541408538818),
 ('bills', 0.704440176486969),
 ('measure', 0.6958035230636597),
 ('passed', 0.690624475479126),
 ('amendment', 0.6846879720687866),
 ('provision', 0.6845567226409912),
 ('plan', 0.6816462874412537),
 ('clinton', 0.6663140058517456)]

In [27]:
# 'cherry'와 유사한 단어의 리스트 반환

model.most_similar('cherry') 

[('peach', 0.688809871673584),
 ('mango', 0.683819055557251),
 ('plum', 0.6684104204177856),
 ('berry', 0.6590359210968018),
 ('grove', 0.658155083656311),
 ('blossom', 0.6503506302833557),
 ('raspberry', 0.6477391719818115),
 ('strawberry', 0.6442098021507263),
 ('pine', 0.6390928626060486),
 ('almond', 0.6379212737083435)]

In [28]:
# 'cherry'와 관련성이 없는 단어의 리스트 반환

model.most_similar(negative = 'cherry')

[('kazushige', 0.48343509435653687),
 ('askerov', 0.4778185784816742),
 ('lakpa', 0.46915265917778015),
 ('ex-gay', 0.4571332633495331),
 ('tadayoshi', 0.4522107243537903),
 ('turani', 0.44810065627098083),
 ('saglam', 0.4469599425792694),
 ('aijun', 0.4435270130634308),
 ('adjustors', 0.44235292077064514),
 ('nyum', 0.4423118233680725)]

In [29]:
# 'woman', 'king'과 유사성이 높으면서 'man'과 관련성이 없는 단어 반환

result = model.most_similar(positive = ['woman', 'king'], negative = ['man'])
print("{}: {:.4f}".format(*result[0]))

queen: 0.7699


In [30]:
# 여러 단어 간 긍정/부정적 관련성

def analogy(x1, x2, y1):
    result = model.most_similar(positive = [y1, x2], negative = [x1])
    return result[0][0]
analogy('australia', 'beer', 'france')

'champagne'

In [31]:
# 새로운 단어 유추

analogy('tall', 'tallest', 'long')

'longest'

In [32]:
# 유사도가 가장 낮은 단어 반환

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

cereal
